---
title: "Data Cleaning"
format:
    html: 
        code-fold: true
---


{{< include overview.qmd >}} 

# Code 

Code for this webpage can be found [here.](https://github.com/dsan-5000/project-dcorc7/blob/main/technical-details/data-cleaning/main.ipynb)

## Importing Python Libraries

The first step in the data cleaning process is to import the appropriate Python libaries. The Python packages that I imported, as well as the reasons for importing them, are as follows:
  
- **pandas:** To work with and store retrieved data into pandas dataframes

- **numpy:** To use the np.nan method in adding NA values when necessary

- **string:** To use punctuation method and enable the removal of built-in punctuation symbols in string values with ease

- **sklearn:** To import a specific list of stopwords with ENGLISH_STOP_WORDS that will be removed from all string values

In [35]:
import pandas as pd
import numpy as np
import string
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

### Loading Raw Data CSV

To begin, I loaded the raw movie data csv into a Pandas dataframe.

In [36]:
movies_df = pd.read_csv("../../data/raw-data/movies.csv")
raw_movies_df = pd.read_csv("../../data/raw-data/movies.csv")

## Removing Unwanted Rows

Movies within the TV Movie, Adventure, Animation, Family, and History genres wre now removed. Movies within these genres have been removed because they have either been deemed too similar to movies in other genres (Action vs Adventure) or they have no common theme (Animation). For example, the movies "Monster House" and "Meet the Robinsons" have vastly different plots. 

In [37]:
# Drop specified movie genres
movies_df = movies_df[~movies_df["Genre"].isin(["TV Movie", "Adventure", "Animation", "Family", "History", "Western", "Documentary"])] 

## Drop Appropriate Values

Next, I have dropped all null values, duplicated movie titles within separate genres, and movies with tv ratings. I dropped all null values, because null values anywhere will interfere with the machine learning models and Exploratory Data Analysis steps. In addition, if a movie does not have a valid overview, then it is incompatiable with my project.

I dropped repeated movies in an interesting way. Because the APIs searched movies by genre and ordered by highest ot lowest revenue and I only assigned one genre per movie, there will inevitably be repeated movies with different asssigned genres. To keep just one movie in these repeated sets and to keep the distribution of genres relatively equal, I randomized the order of the movies and kept the title that occurred first out of the movie repeats.

The last removal I performed was for rows with tv ratings. I dropped these values because I only want to focus on movies, not tv shows.

In [38]:
# Drop rows with NA values
#movies_df = movies_df.dropna(subset = ["Overview"])
movies_df = movies_df.dropna()

# Drop duplicate movies  
print(f"Duplicate Movie Count: {movies_df["Title"].duplicated().sum()}")  

movies_df = movies_df.sample(frac = 1, random_state = 123).reset_index(drop = True)
movies_df = movies_df.drop_duplicates(subset = "Title", keep = "first")

# Drop movies with tv rating
movies_df = movies_df[~movies_df["Age_Rating"].isin(["TV-MA", "TV-PG", "Approved", "Unrated", "TV-14", "TV-Y7", "TV-G"])]

Duplicate Movie Count: 1270


## Ensure Equal Distribution of Movies in Each Genre 

To avoid bias in some of the machine learning models, I wanted to ensure that each genre is equally represented in the dataset. To accomplish this, I set a target value of movies, looped through movies in each genre, sampled movies in genres in which the target value was exceeded, and removed movies not appearing within the random sample. After performing this method, each genre either contained 150 or just under 150 movies.

In [39]:
# Ensure an equal distribution of movies per genre
target_count = 150
genres = movies_df["Genre"].unique().tolist()

for genre in genres:
    # If there are more movies in the genre than the target number, then sample the movies and remove them
    genre_df = movies_df[movies_df["Genre"] == genre]
    if len(genre_df) > target_count:
        # Sample movies of the specified genre
        genre_df = genre_df.sample(n = target_count, random_state = 2024)

        # Remove all movies currently assigned to the specified genre
        movies_df = movies_df[movies_df["Genre"] != genre]

        # Add back in the sampled movies
        movies_df = pd.concat([movies_df, genre_df], axis = 0, ignore_index = True)

## Lowercase Text Data and Convert to String Values

Following the restructuring of the data, I converted values in each text-based column to string values and made each character lowercase. This will help with NLP methods later on.

In [40]:
# Convert text data to strings and lowercase
movies_df["IMDB_ID"] = movies_df["IMDB_ID"].astype(str).str.lower()
movies_df["Title"] = movies_df["Title"].astype(str).str.lower()
movies_df["Overview"] = movies_df["Overview"].astype(str).str.lower()
movies_df["Genre"] = movies_df["Genre"].astype(str).str.lower()
movies_df["Director"] = movies_df["Director"].astype(str).str.lower()
movies_df["Age_Rating"] = movies_df["Age_Rating"].astype(str).str.lower()
movies_df["Actors"] = movies_df["Actors"].astype(str).str.lower()
movies_df["Keywords"] = movies_df["Keywords"].astype(str).str.lower()
movies_df["Awards"] = movies_df["Awards"].astype(str).str.lower()

## Remove Punctuation

Similar to the string and lowercase conversion, removing puncuation from columns that have it is necessary for accurate NLP operations. Removing puncuation was accomplished with the "string" Python library's "punctuation" method.

In [41]:
# Remove Punctuation from string values, except replaces dashes (-) with spaces
translation_table = str.maketrans("-", " ", string.punctuation.replace("-", ""))
movies_df["Overview"] = movies_df["Overview"].str.translate(translation_table)
movies_df["Title"] = movies_df["Title"].str.translate(translation_table)
movies_df["Actors"] = movies_df["Actors"].str.translate(translation_table)
movies_df["Keywords"] = movies_df["Keywords"].str.translate(translation_table)

## Remove Stopwords

One last text-based data cleaning method was to remove stopwords from string values. Stopwords are words that often appear in text but don't carry much meaning on their own. This process was accomplished using "sklearn.feature_extraction's" ENGLISH_STOP_WORDS method. This method provides a built-in list of stopwords that are common among text-based removal operations. I combined this list with a custom list of stopwords to broaden the amount of unnecessary words used in this step.

In [42]:
# Remove stopwords from the Overview column
custom_stopwords = ["one", "hannah", "story", "old", "new"]
stopword_list = custom_stopwords + list(ENGLISH_STOP_WORDS)
movies_df["Overview"] = movies_df["Overview"].apply(lambda text: " ".join(word for word in text.split() if word not in stopword_list) if isinstance(text, str) else text)

## Convert Data Types

Now that all string values were dealt with, I converted non-string dataypes. Using pd.numeric, I converted values in the "Year", "Metascore_Rating", and "IMDB_Rating" columns to integers. I also removed percentage symbols from values in the "Rotten_Tomatoes_Rating" column and converted them to integers. Values in the "Actors" and "Keywords" columns were split from their list formats and converted to strings. "Release_Date" and "Release_Month" values were converted to their respective date values. 

In [43]:
# Convert columns to specified data types
movies_df["Year"] = pd.to_numeric(movies_df["Year"], errors = "coerce").astype("Int64")
movies_df["Runtime"] = movies_df["Runtime"].str.extract(r"(\d+)").astype("Int64")
movies_df["Metascore_Rating"] = pd.to_numeric(movies_df["Metascore_Rating"], errors = "coerce").astype("Int64") / 10
movies_df["Rotten_Tomatoes_Rating"] = movies_df["Rotten_Tomatoes_Rating"].str.rstrip('%').astype("Int64") / 10
movies_df["IMDB_Rating"] = pd.to_numeric(movies_df["IMDB_Rating"], errors = "coerce")
movies_df["Actors"] = movies_df["Actors"].str.split(", ")
movies_df["Keywords"] = movies_df["Keywords"].str.split(", ")
movies_df["TMDB_Rating"] = movies_df["TMDB_Rating"].round(1)
movies_df["Release_Date"] = pd.to_datetime(movies_df["Release_Date"], errors = "coerce")
movies_df["Release_Month"] = movies_df["Release_Date"].dt.month_name()

## Create Additional Columns

Following datatype conversions, I created 7 new columns: Oscar_Wins, Oscar_Nominations, Won_Award, Return_On_Investment, Average_Rating, Budget_Normalized, and Revenue_Normalized. The Oscar_Wins and Oscar_Nominations columns were created by altering the original Awards column to extract how many academy ward wins and nominations each movies earned. A new Won_Award column, occupied with boolean values, was then created based on whether values in the Oscar_Wins column were non-zero. The Return_On_Investment column as calculated by dividing the Revenue values by the Budget values. The Average_Rating column was calculated to provide a mean rating among all provided score systems. Lastly, Z-Score normalization was applied to both the budget and revenue columns to make the two new Budget_Normalized and Revenue_Normalized columns. Normalization is important because it ensures that features are scaled appropriately for machine learning tasks and removes bias from extreme dollar values.

In [44]:
# Alter Awards column to only include oscar counts and create new columns based on nominations vs wins
movies_df["Awards"] = movies_df["Awards"].apply(lambda x: x.split("oscar")[0] + "oscar" if "oscar" in str(x) else np.nan)
movies_df["Oscar_Wins"] = movies_df["Awards"].apply(lambda x: int(x.split("won")[1].split("oscar")[0].strip()) if "won" in str(x) else 0)
movies_df["Oscar_Nominations"] = movies_df["Awards"].apply(lambda x: int(x.split("nominated for")[1].split("oscar")[0].strip()) if "nominated" in str(x) else 0)
movies_df = movies_df.drop(columns = ["Awards"])

# Add boolean column for oscar win or not
movies_df["Won_Award"] = movies_df["Oscar_Wins"] != 0

# Return_On_Investment column calculated by dividing movie revenue by budget
movies_df["Return_On_Investment"] = movies_df["Revenue"] / movies_df["Budget"]

# Average_Rating column calculated by obatining the mean of all provided rating scores
movies_df["Average_Rating"] = movies_df[["Metascore_Rating", "IMDB_Rating", "Rotten_Tomatoes_Rating", "TMDB_Rating"]].mean(axis=1)


# Z-Score normalization for Budget and Revenue columns
movies_df["Budget_Normalized"] = (movies_df["Budget"] - movies_df["Budget"].mean()) / movies_df["Budget"].std()
movies_df["Revenue_Normalized"] = (movies_df["Revenue"] - movies_df["Revenue"].mean()) / movies_df["Revenue"].std()

## Remove Inf and -Inf Values

Some columns had Inf, values, so I removed these occurances as well. Similar to null values, Inf values will not be plotted in the EDA step, nor work with some machine learning models.

In [45]:
# Replace infinity values with NaN, then remove NaN
movies_df.replace([np.inf, -np.inf], np.nan, inplace = True)
movies_df.dropna(inplace = True)

## Rearrange Columns

Columns were then rearranged to promote the readability of the dataframe. I wanted the columns to be ordered in a logical way. For example, I wanted "IMDB_ID", "Title", and and "Release_Date" to be some of the first columns displayed.

In [46]:
# Rearrange Columns in movies_df 
movies_df = movies_df[["IMDB_ID", "Title", "Year", "Release_Date", "Release_Month", "Age_Rating", 
                    "Overview", "Keywords", "Genre", "Director", "Actors", "Runtime", "Metascore_Rating", 
                    "IMDB_Rating", "Rotten_Tomatoes_Rating", "TMDB_Rating", "Average_Rating", "Won_Award", 
                    "Oscar_Wins", "Oscar_Nominations", "Budget", "Budget_Normalized", "Revenue", "Revenue_Normalized", 
                    "Return_On_Investment", "Popularity"]]

## Save Dataframe to CSV

Lastly, the dataframe was saved as a csv file and ready for analysis.

In [47]:
movies_df.to_csv("../../data/processed-data/movies_cleaned.csv", index = False)

## Previewing the Cleaned Dataframe

After cleaning the data through the various processes, we can view the total movie count, shape of the finalized dataframe, counts per genre, and total data for first 5 movies.

In [48]:
print(f"Total Movie Count: {len(movies_df)}\n")

print(f"Clean Datatset Shape: {movies_df.shape}\n")

print(f"{movies_df["Genre"].value_counts()}\n")

Total Movie Count: 1699

Clean Datatset Shape: (1699, 26)

Genre
horror             149
science fiction    149
fantasy            148
crime              148
romance            147
comedy             147
action             147
mystery            143
thriller           140
music              129
war                127
drama              125
Name: count, dtype: int64



### Dataset Before Cleaning:

In [49]:
pd.set_option("display.max_columns", None)
selected_movies = ["tt1853728", "tt0816692", "tt0325980", "tt1392190", "tt0477348"]
raw_display_df = raw_movies_df[raw_movies_df["IMDB_ID"].isin(selected_movies)].drop_duplicates(subset = "IMDB_ID")
raw_display_df.head(5)

,IMDB_ID,Title,Release_Date,Age_Rating,Overview,Popularity,Genre,TMDB_Rating,Budget,Revenue,Keywords,Year,Director,Actors,Runtime,Awards,Metascore_Rating,IMDB_Rating,Rotten_Tomatoes_Rating
92,tt0325980,Pirates of the Caribbean: The Curse of the Bla...,2003-07-09,PG-13,After Port Royal is attacked and pillaged by a...,122.691,Action,7.806,140000000,655011224,"['blacksmith', 'east india company', 'gold', '...",2003,Gore Verbinski,"Johnny Depp, Geoffrey Rush, Orlando Bloom",143 min,Nominated for 5 Oscars. 38 wins & 104 nominati...,63.0,8.1,79%
216,tt1392190,Mad Max: Fury Road,2015-05-13,R,An apocalyptic story set in the furthest reach...,78.602,Action,7.600,150000000,378858340,"['rescue', 'future', 'australia', 'chase', 'dy...",2015,George Miller,"Tom Hardy, Charlize Theron, Nicholas Hoult",120 min,Won 6 Oscars. 245 wins & 233 nominations total,90.0,8.1,97%
514,tt0816692,Interstellar,2014-11-05,PG-13,The adventures of a group of explorers who mak...,245.220,Adventure,8.400,165000000,701729206,"['rescue', 'future', 'spacecraft', 'race again...",2014,Christopher Nolan,"Matthew McConaughey, Anne Hathaway, Jessica Ch...",169 min,Won 1 Oscar. 44 wins & 148 nominations total,74.0,8.7,73%
1725,tt0477348,No Country for Old Men,2007-06-13,R,"Llewelyn Moss stumbles upon dead bodies, $2 mi...",56.384,Crime,7.900,25000000,171627166,"['vietnam veteran', 'sheriff', 'trailer park',...",2007,"Ethan Coen, Joel Coen","Tommy Lee Jones, Javier Bardem, Josh Brolin",122 min,Won 4 Oscars. 164 wins & 139 nominations total,92.0,8.2,93%
2463,tt1853728,Django Unchained,2012-12-25,R,"With the help of a German bounty hunter, a fre...",137.546,Drama,8.200,100000000,425368238,"['rescue', 'friendship', 'bounty hunter', 'tex...",2012,Quentin Tarantino,"Jamie Foxx, Christoph Waltz, Leonardo DiCaprio",165 min,Won 2 Oscars. 58 wins & 158 nominations total,81.0,8.5,87%


### Dataset After Cleaning:

In [50]:
pd.set_option("display.max_columns", None)
selected_movies = ["tt1853728", "tt0816692", "tt0325980", "tt1392190", "tt0477348"]
cleaned_display_df = movies_df[movies_df["IMDB_ID"].isin(selected_movies)]
cleaned_display_df.head(5)

,IMDB_ID,Title,Year,Release_Date,Release_Month,Age_Rating,Overview,Keywords,Genre,Director,Actors,Runtime,Metascore_Rating,IMDB_Rating,Rotten_Tomatoes_Rating,TMDB_Rating,Average_Rating,Won_Award,Oscar_Wins,Oscar_Nominations,Budget,Budget_Normalized,Revenue,Revenue_Normalized,Return_On_Investment,Popularity
222,tt1392190,mad max fury road,2015,2015-05-13,May,r,apocalyptic set furthest reaches planet stark ...,[rescue future australia chase dystopia post a...,action,george miller,[tom hardy charlize theron nicholas hoult],120,9.0,8.1,9.7,7.6,8.6,True,6,0,150000000,1.491689,378858340,0.579184,2.525722,78.602
274,tt0477348,no country for old men,2007,2007-06-13,June,r,llewelyn moss stumbles dead bodies 2 million h...,[vietnam veteran sheriff trailer park based on...,thriller,"ethan coen, joel coen",[tommy lee jones javier bardem josh brolin],122,9.2,8.2,9.3,7.9,8.65,True,4,0,25000000,-0.605078,171627166,-0.196899,6.865087,56.384
492,tt0325980,pirates of the caribbean the curse of the blac...,2003,2003-07-09,July,pg-13,port royal attacked pillaged mysterious pirate...,[blacksmith east india company gold exotic isl...,action,gore verbinski,[johnny depp geoffrey rush orlando bloom],143,6.3,8.1,7.9,7.8,7.525,False,0,5,140000000,1.323948,655011224,1.613379,4.678652,122.691
515,tt0816692,interstellar,2014,2014-11-05,November,pg-13,adventures group explorers make use newly disc...,[rescue future spacecraft race against time ar...,drama,christopher nolan,[matthew mcconaughey anne hathaway jessica cha...,169,7.4,8.7,7.3,8.4,7.95,True,1,0,165000000,1.743301,701729206,1.788339,4.252904,245.220
553,tt1853728,django unchained,2012,2012-12-25,December,r,help german bounty hunter freed slave sets res...,[rescue friendship bounty hunter texas slavery...,drama,quentin tarantino,[jamie foxx christoph waltz leonardo dicaprio],165,8.1,8.5,8.7,8.2,8.375,True,2,0,100000000,0.652982,425368238,0.753364,4.253682,137.546



{{< include closing.qmd >}} 